# Speech denoising using RNN

In [1]:
#import libraries
import librosa
import numpy
import os
import numpy as np
import tensorflow as tf
import statistics

Make sure the timit-homework folder is inside the current working directory or set path accordingly "path/timit-homework.."

In [2]:
#set path
path = os.getcwd() 

Load training files located in /tr folder

In [3]:
#load train files
X_tr_load = []
S_tr_load = []
N_tr_load = []

X_tr_stft = []
S_tr_stft = []
N_tr_stft = []

X_tr_abs = []
S_tr_abs = []
N_tr_abs = []

X_tr_len = []
S_tr_len = []
N_tr_len = []
for i in os.listdir(path+'\\timit-homework\\tr'):
    if 'trx' in i:
        s, sr=librosa.load("timit-homework/tr/"+i, sr=None)
        X_tr_load.append(s)
        X=librosa.stft(s, n_fft=1024, hop_length=512)
        X_tr_stft.append(X)
        X_tr_len.append(X.shape[1])
        X_abs=np.abs(np.transpose(X))
        X_tr_abs.append(X_abs)
    elif 'trn' in i:
        s, sr=librosa.load("timit-homework/tr/"+i, sr=None)
        N_tr_load.append(s)
        N=librosa.stft(s, n_fft=1024, hop_length=512)
        N_tr_stft.append(N)
        N_tr_len.append(N.shape[1])
        N_abs=np.abs(np.transpose(N))
        N_tr_abs.append(N_abs)
    elif 'trs' in i:
        s, sr=librosa.load("timit-homework/tr/"+i, sr=None)
        S_tr_load.append(s)
        S=librosa.stft(s, n_fft=1024, hop_length=512)
        S_tr_stft.append(S)
        S_tr_len.append(S.shape[1])
        S_abs=np.abs(np.transpose(S))
        S_tr_abs.append(S_abs)
print(len(S_tr_abs),len(X_tr_abs),len(N_tr_abs))


1200 1200 1200


Load validation files located in /v folder

In [4]:
#Load validation files
V_x_load = []
V_s_load = []
V_n_load = []

V_x_stft = []
V_s_stft = []
V_n_stft = []

V_x_abs = []
V_s_abs = []
V_n_abs = []

V_x_len = []
V_s_len = []
V_n_len = []
for i in os.listdir(path +'\\timit-homework\\v'):
    if 'vx' in i:
        s, sr=librosa.load("timit-homework/v/"+i, sr=None)
        V_x_load.append(s)
        X=librosa.stft(s, n_fft=1024, hop_length=512)
        V_x_stft.append(X)
        V_x_len.append(X.shape[1])
        X_abs=np.abs(np.transpose(X))
        V_x_abs.append(X_abs)
    elif 'vn' in i:
        s, sr=librosa.load("timit-homework/v/"+i, sr=None)
        V_n_load.append(s)
        N=librosa.stft(s, n_fft=1024, hop_length=512)
        V_n_stft.append(N)
        V_n_len.append(X.shape[1])
        N_abs=np.abs(np.transpose(N))
        V_n_abs.append(N_abs)
    elif 'vs' in i:
        s, sr=librosa.load("timit-homework/v/"+i, sr=None)
        V_s_load.append(s)
        S=librosa.stft(s, n_fft=1024, hop_length=512)
        V_s_stft.append(S)
        V_s_len.append(X.shape[1])
        S_abs=np.abs(np.transpose(S))
        V_s_abs.append(S_abs)
print(len(V_x_abs),len(V_s_abs),len(V_n_abs))

1200 1200 1200


Pad the files to feed into RNN with size 200(chose 200 to keep it consistent across the network because the max length in test files is 195) and calculate IBM

In [5]:
#PAD validation data
M_val_abs = []
maxVS=0
maxVN=0
maxVX=0

#find max length of signal to pad with
for i in range(1200):
    maxVS=max(maxVS,V_s_abs[i].shape[0])
    maxVX=max(maxVX,V_x_abs[i].shape[0])
    maxVN=max(maxVN,V_n_abs[i].shape[0])
    #print(maxVS,maxVX,maxVN)

#replacing max values with 200 to match with test data, since max of test is 195 not 178 and it was causing an error later on.    
maxVS=200
maxVN=200
maxVX=200


#pad each signal upto the max length
for i in range(1200):
    V_x_abs[i]=np.pad(V_x_abs[i],((0,maxVX-V_x_abs[i].shape[0]),(0,0)),'constant')
    V_s_abs[i]=np.pad(V_s_abs[i],((0,maxVS-V_s_abs[i].shape[0]),(0,0)),'constant')
    V_n_abs[i]=np.pad(V_n_abs[i],((0,maxVN-V_n_abs[i].shape[0]),(0,0)),'constant')

#pad each signal upto the max length
for i in range(1200):
    V_x_stft[i]=np.pad(V_x_stft[i].T,((0,maxVX-V_x_stft[i].shape[1]),(0,0)),'constant')
    V_s_stft[i]=np.pad(V_s_stft[i].T,((0,maxVS-V_s_stft[i].shape[1]),(0,0)),'constant')
    V_n_stft[i]=np.pad(V_n_stft[i].T,((0,maxVN-V_n_stft[i].shape[1]),(0,0)),'constant')

#create IBM matrix
for i in range(1200):
    M_val_abs.append(1*(V_s_abs[i]>V_n_abs[i]))

In [6]:
#PAD Training data 
M_tr_abs=[]
maxS=0
maxN=0
maxX=0

#find max length of signal to pad with
for i in range(1200):
    maxS=max(maxS,S_tr_abs[i].shape[0])
    maxX=max(maxX,X_tr_abs[i].shape[0])
    maxN=max(maxN,N_tr_abs[i].shape[0])
#replacing max values with 200 to match with test data, since max of test is 195 not 178 and it was causing an error later on.    
maxS=200
maxX=200
maxN=200

#pad each signal upto the max length
for i in range(1200):
    X_tr_abs[i]=np.pad(X_tr_abs[i],((0,maxX-X_tr_abs[i].shape[0]),(0,0)),'constant')
    S_tr_abs[i]=np.pad(S_tr_abs[i],((0,maxS-S_tr_abs[i].shape[0]),(0,0)),'constant')
    N_tr_abs[i]=np.pad(N_tr_abs[i],((0,maxN-N_tr_abs[i].shape[0]),(0,0)),'constant')

#pad each signal upto the max length
for i in range(1200):
    X_tr_stft[i]=np.pad(X_tr_stft[i].T,((0,maxX-X_tr_stft[i].shape[1]),(0,0)),'constant')
    S_tr_stft[i]=np.pad(S_tr_stft[i].T,((0,maxS-S_tr_stft[i].shape[1]),(0,0)),'constant')
    N_tr_stft[i]=np.pad(N_tr_stft[i].T,((0,maxN-N_tr_stft[i].shape[1]),(0,0)),'constant')
    
#create IBM matrix
for i in range(1200):
    M_tr_abs.append(1*(S_tr_abs[i]>N_tr_abs[i]))

In [215]:
np.where(M_tr_abs[0]==1)

(array([ 0,  0,  0, ..., 62, 62, 63], dtype=int64),
 array([ 71, 111, 178, ...,  28, 263,  66], dtype=int64))

Network architecture:

For the network I chose a dynamic rnn cell consisting of one LSTM cell with 513 hidden units. Followed with an output dense layer with sigmoid activation.

I tried implementing multilayer RNN and bidirectional LSTM as well but the results weren't as good as compared to the above architecture.

In [198]:
#https://github.com/tensorflow/nmt
#https://www.tensorflow.org/api_docs/python/tf/compat/v1/nn/dynamic_rnn
tf.reset_default_graph()
keep_prob = tf.placeholder(tf.float32)
seq_len= tf.placeholder(tf.int32)
#place holder for input [batchsize,padding length(kept none to accomodate any shape),513]
X = tf.placeholder(tf.float32, [None,200,513])
print(X.shape)

#Dynamic RNN Layer with one LSTM cell of 513 units
rnn_layer_output, rnn_layer_states  = tf.nn.dynamic_rnn(cell = tf.nn.rnn_cell.DropoutWrapper(\
         cell = tf.nn.rnn_cell.LSTMCell(num_units = 513, initializer = tf.contrib.layers.xavier_initializer()),\
                                                                 output_keep_prob=keep_prob),\
                                                         inputs = X, dtype = tf.float32, sequence_length=seq_len)
rnn_layer_output = tf.nn.relu(rnn_layer_output)
#rnn_layer_output, rnn_layer_states  = tf.nn.dynamic_rnn(cell = tf.nn.rnn_cell.LSTMCell(num_units = 513),\
 #                                                        inputs = X, dtype = tf.float32)
y = tf.layers.dense(rnn_layer_output, 513, activation=tf.nn.sigmoid)
print(y.shape)

#Placeholder for output
y_ = tf.placeholder(tf.float32, [None,200,513])



#Trial and error of other networks


# rnn_layer_output_rev, rnn_layer_states_rev  = tf.nn.dynamic_rnn(cell = tf.nn.rnn_cell.DropoutWrapper(\
#         cell = tf.nn.rnn_cell.LSTMCell(num_units = 513, initializer = tf.contrib.layers.xavier_initializer()),\
#                                                                 output_keep_prob=keep_prob),\
#                                                         inputs = rev_X, dtype = tf.float32)#, sequence_length=seq_len)

#output
# create 2 LSTMCells
# rnn_layers = [tf.compat.v1.nn.rnn_cell.LSTMCell(size) for size in [256, 513]]

# # create a RNN cell composed sequentially of a number of RNNCells
# multi_rnn_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell(rnn_layers)

# # 'outputs' is a tensor of shape [batch_size, max_time, 256]
# # 'state' is a N-tuple where N is the number of LSTMCells containing a
# # tf.nn.rnn_cell.LSTMStateTuple for each cell
# rnn_layer_output, state = tf.compat.v1.nn.dynamic_rnn(cell=multi_rnn_cell,
#                                    inputs=X,
#                                    dtype=tf.float32)
#concat_rnn = tf.concat([rnn_layer_output, rnn_layer_output_rev])
#h2 = tf.layers.dense(rnn_layer_output, 256, activation = tf.nn.relu)


(?, 200, 513)
(?, 200, 513)


Loss function used is mean squared error and optimizer is Adam with learning rate of 0.008. The network was trained for 50 epochs with a batch size of 10.

In [199]:
error = tf.losses.mean_squared_error(y_,y)
optimizer = tf.train.AdamOptimizer(0.008).minimize(error)
loss_log = []
val_loss=[]
init = tf.global_variables_initializer()
epochs = 5
batch_size = 10

Run the network

In [216]:
X_tr_abs[0:10]

[array([[3.8563597e-01, 9.4302349e-02, 5.2421097e-02, ..., 1.3693520e-01,
         5.8561459e-04, 7.5867265e-02],
        [6.0343511e-02, 2.7758509e-01, 6.3259554e-01, ..., 6.0125597e-02,
         5.1691905e-02, 8.6459212e-02],
        [1.1737749e-02, 2.6299518e-01, 5.4270428e-01, ..., 9.1117688e-02,
         4.9656875e-02, 1.3668190e-02],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00]], dtype=float32),
 array([[0.04266779, 0.01570266, 0.00084584, ..., 0.00162074, 0.00155778,
         0.00194192],
        [0.00415536, 0.0113495 , 0.07119855, ..., 0.00782664, 0.0040458 ,
         0.0024524 ],
        [0.00355178, 0.05958127, 0.1121546 , ..., 0.01123079, 0.00444199,
         0.00300022]

In [200]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)


sess_a3_part2 = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess_a3_part2.run(init)

for epoch in range(epochs):
    temp_loss = 0
    temp_loss2 = 0
    for i in range(0,1200,batch_size):
        loss,_ = sess_a3_part2.run([error, optimizer], feed_dict = {X: np.array(X_tr_abs[i:i+batch_size]),\
                                                                    y_: np.array(M_tr_abs[i:i+batch_size]),\
                                                                   keep_prob:0.8,seq_len:X_tr_len[i:i+batch_size]})
        valloss = sess_a3_part2.run(error, feed_dict = {X: np.array(V_x_abs[i:i+batch_size]),\
                                                        y_: np.array(M_val_abs[i:i+batch_size]),\
                                                       keep_prob:0.8,seq_len:V_x_len[i:i+batch_size]})
        temp_loss+=loss
        temp_loss2+= valloss
    loss_log.append(temp_loss/120)
    val_loss.append(temp_loss2/120)
    print("Epoch:",epoch+1,"Train loss:",temp_loss/120, "Validation loss:",temp_loss2/120)   

Epoch: 1 Train loss: 0.18147837954262894 Validation loss: 0.17783689176042874
Epoch: 2 Train loss: 0.11656482567389806 Validation loss: 0.1132462236409386
Epoch: 3 Train loss: 0.09477764669184884 Validation loss: 0.0924822241688768
Epoch: 4 Train loss: 0.08576707194248835 Validation loss: 0.08412005724385381
Epoch: 5 Train loss: 0.08020474212244152 Validation loss: 0.07920855892201265


Run the network to calculate output on validation set.

In [217]:
#output on validation set
# val_output = []
pred = np.zeros((0,200,513))
for i in range(0,1200,batch_size):
    op = sess_a3_part2.run(y,feed_dict = {X: np.array(V_x_abs[i:i+batch_size]), keep_prob:0.8, seq_len:V_x_len[i:i+batch_size]})
#     val_output.append(op)
    pred = np.append(pred, op,0)


In [218]:
pred.shape

(1200, 200, 513)

Converting output to masks

In [202]:
#Converting output to a list of 2d data, where data is converted to binary mask where values > 0.5 are considered.
final = []
for i in range(len(val_output)):
     for j in range(batch_size):
        final.append(1*(val_output[i][j]>0.5))

SNR on Validation set

In [219]:
if not os.path.exists('val_outputs'):
    os.mkdir('val_outputs')

In [220]:
#Calculate SNR on validation set
SNR=[]
for i in range(1200):
    c
    clean_istft = librosa.istft(temp.T,win_length=1024, hop_length = 512, length = V_x_load[i].shape[0])
    librosa.output.write_wav("val_outputs/val_clean_"+str(i)+".wav", clean_istft, sr=16000)
    num = (np.sum(V_s_load[i]*V_s_load[i]))
    den = np.sum(np.square(V_s_load[i]-clean_istft))
    SNR.append(10*np.log10(num/den))
    print("Validation file",i+1,"SNR:",10*np.log10(num/den))

Validation file 1 SNR: 11.92217230796814
Validation file 2 SNR: 8.38738203048706
Validation file 3 SNR: 8.934089541435242
Validation file 4 SNR: 0.9049903601408005
Validation file 5 SNR: 2.9621869325637817
Validation file 6 SNR: 7.288333177566528
Validation file 7 SNR: 3.49418967962265
Validation file 8 SNR: 7.93589174747467
Validation file 9 SNR: 3.3456453680992126
Validation file 10 SNR: 2.294284999370575
Validation file 11 SNR: 8.846185803413391
Validation file 12 SNR: 3.2219520211219788
Validation file 13 SNR: 10.143791437149048
Validation file 14 SNR: 2.9721489548683167
Validation file 15 SNR: 2.687869071960449
Validation file 16 SNR: 8.28476369380951
Validation file 17 SNR: 3.9537376165390015
Validation file 18 SNR: 6.500986218452454
Validation file 19 SNR: 3.197188079357147
Validation file 20 SNR: 3.4619414806365967
Validation file 21 SNR: 4.522120952606201
Validation file 22 SNR: 3.37330162525177
Validation file 23 SNR: 8.262532949447632
Validation file 24 SNR: 0.88549315929412

Validation file 198 SNR: 9.640617370605469
Validation file 199 SNR: 5.393396019935608
Validation file 200 SNR: 3.683292269706726
Validation file 201 SNR: 12.652809619903564
Validation file 202 SNR: 5.528522729873657
Validation file 203 SNR: 9.451591968536377
Validation file 204 SNR: 2.4504369497299194
Validation file 205 SNR: 6.600739359855652
Validation file 206 SNR: 19.06926393508911
Validation file 207 SNR: 8.544510006904602
Validation file 208 SNR: 11.183680295944214
Validation file 209 SNR: 6.041696667671204
Validation file 210 SNR: 2.6279765367507935
Validation file 211 SNR: 11.565353870391846
Validation file 212 SNR: 3.8362297415733337
Validation file 213 SNR: 4.797013998031616
Validation file 214 SNR: 4.002668857574463
Validation file 215 SNR: 4.65117335319519
Validation file 216 SNR: 10.900572538375854
Validation file 217 SNR: 6.876987814903259
Validation file 218 SNR: 9.580880403518677
Validation file 219 SNR: 6.327502131462097
Validation file 220 SNR: 3.7972626090049744
Vali

Validation file 403 SNR: 7.260732054710388
Validation file 404 SNR: 4.693471789360046
Validation file 405 SNR: 9.426098465919495
Validation file 406 SNR: 8.851923942565918
Validation file 407 SNR: 7.018499970436096
Validation file 408 SNR: 5.8666664361953735
Validation file 409 SNR: 4.608096778392792
Validation file 410 SNR: 1.8047478795051575
Validation file 411 SNR: 14.404231309890747
Validation file 412 SNR: 2.6634448766708374
Validation file 413 SNR: 12.245148420333862
Validation file 414 SNR: 3.630208373069763
Validation file 415 SNR: 6.298477649688721
Validation file 416 SNR: 8.17279040813446
Validation file 417 SNR: 6.443523168563843
Validation file 418 SNR: 9.496830701828003
Validation file 419 SNR: 6.379925012588501
Validation file 420 SNR: 1.8500646948814392
Validation file 421 SNR: 11.383622884750366
Validation file 422 SNR: 3.005360960960388
Validation file 423 SNR: 10.702664852142334
Validation file 424 SNR: 2.5683966279029846
Validation file 425 SNR: 4.127042591571808
Val

Validation file 598 SNR: 5.16901969909668
Validation file 599 SNR: 5.26067316532135
Validation file 600 SNR: 1.0984068363904953
Validation file 601 SNR: 12.692904472351074
Validation file 602 SNR: 3.696102797985077
Validation file 603 SNR: 6.596116423606873
Validation file 604 SNR: 2.7033627033233643
Validation file 605 SNR: 7.532475590705872
Validation file 606 SNR: 10.12433648109436
Validation file 607 SNR: 5.017910599708557
Validation file 608 SNR: 11.384114027023315
Validation file 609 SNR: 6.659892201423645
Validation file 610 SNR: 3.0295979976654053
Validation file 611 SNR: 12.096284627914429
Validation file 612 SNR: 3.0770766735076904
Validation file 613 SNR: 6.610775589942932
Validation file 614 SNR: 0.9486763924360275
Validation file 615 SNR: 6.684028506278992
Validation file 616 SNR: 11.261389255523682
Validation file 617 SNR: 5.032778978347778
Validation file 618 SNR: 10.657156705856323
Validation file 619 SNR: 6.666291952133179
Validation file 620 SNR: 0.6571444123983383
Va

Validation file 806 SNR: 10.318100452423096
Validation file 807 SNR: 5.514426231384277
Validation file 808 SNR: 9.3572598695755
Validation file 809 SNR: 7.08573043346405
Validation file 810 SNR: 1.4401499927043915
Validation file 811 SNR: 13.465814590454102
Validation file 812 SNR: 2.4639424681663513
Validation file 813 SNR: 5.311374068260193
Validation file 814 SNR: 2.3129531741142273
Validation file 815 SNR: 3.717607259750366
Validation file 816 SNR: 9.742386937141418
Validation file 817 SNR: 2.246093899011612
Validation file 818 SNR: 3.215957283973694
Validation file 819 SNR: 4.944014549255371
Validation file 820 SNR: 5.378711819648743
Validation file 821 SNR: 9.922646880149841
Validation file 822 SNR: 2.098540812730789
Validation file 823 SNR: 8.382689356803894
Validation file 824 SNR: 2.7045896649360657
Validation file 825 SNR: 6.258036494255066
Validation file 826 SNR: 7.985674142837524
Validation file 827 SNR: 4.63390052318573
Validation file 828 SNR: 5.361257791519165
Validatio

Validation file 1013 SNR: 1.476399451494217
Validation file 1014 SNR: 0.704306960105896
Validation file 1015 SNR: 7.394968271255493
Validation file 1016 SNR: 12.63249397277832
Validation file 1017 SNR: 5.734121799468994
Validation file 1018 SNR: 6.9061267375946045
Validation file 1019 SNR: 6.088452339172363
Validation file 1020 SNR: 1.0752755403518677
Validation file 1021 SNR: 13.24485182762146
Validation file 1022 SNR: 3.3293601870536804
Validation file 1023 SNR: 5.053514838218689
Validation file 1024 SNR: 2.2420595586299896
Validation file 1025 SNR: 5.898845791816711
Validation file 1026 SNR: 15.597715377807617
Validation file 1027 SNR: 5.865328311920166
Validation file 1028 SNR: 9.208212494850159
Validation file 1029 SNR: 9.040529131889343
Validation file 1030 SNR: 3.360181152820587
Validation file 1031 SNR: 14.449384212493896
Validation file 1032 SNR: 2.8714656829833984
Validation file 1033 SNR: 3.955671489238739
Validation file 1034 SNR: 4.21692818403244
Validation file 1035 SNR: 

In [221]:
#mean of SNR of validation set
print("Mean of SNR:",statistics.mean(SNR))

Mean of SNR: 6.6621337432879955


Load testing data:

In [222]:
#load test files
test_load = []
test_stft = []
test_abs = []
test_len = []
for i in os.listdir(os.getcwd()+'\\timit-homework\\te'):
    s, sr=librosa.load("timit-homework/te/"+i, sr=None)
    test_load.append(s)
    X_signal = librosa.stft(s, n_fft=1024, hop_length=512)
    test_stft.append(X_signal)
    test_len.append(X_signal.shape[1])
    X_abs=np.abs(np.transpose(X_signal))
    test_abs.append(X_abs)
#print(len(testabs))

In [223]:
max_test=0
#find max length of signal to pad with
for i in range(len(test_abs)):
    max_test=max(max_test,test_abs[i].shape[0])
max_test=200

Pad test data:

In [224]:
#pad each signal upto the max length
for i in range(400):
    test_abs[i]=np.pad(test_abs[i],((0,max_test-test_abs[i].shape[0]),(0,0)),'constant')

#pad each signal upto the max length
for i in range(400):
    test_stft[i]=np.pad(test_stft[i].T,((0,max_test-test_stft[i].shape[1]),(0,0)),'constant')

Run the network on testing data

In [225]:
#output on test set
# test_output = []
pred = np.zeros((0,200,513))
for i in range(0,400,batch_size):
    op_test = sess_a3_part2.run(y,feed_dict = {X: np.array(test_abs[i:i+batch_size]), keep_prob:0.8, seq_len:test_len[i:i+batch_size]})
#     test_output.append(op)
    pred = np.append(pred, op,0)


In [153]:
print(float(abs(2.04052478e-01+0.00000000e+00j)))

0.204052478


In [188]:
print(test_stft[0][199])

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.

In [127]:
print(np.array(test_output).shape)

(40, 10, 200, 513)


In [128]:
np.save('pred_test.npy',np.array(test_output))

create directory to save test outputs

In [210]:
if not os.path.exists('test_outputs'):
    os.mkdir('test_outputs')

convert output to masks:

In [211]:
#Converting output to a list of 2d data, where data is converted to binary mask where values > 0.5 are considered.
final_test = []
for i in range(len(test_output)):
     for j in range(batch_size):
        final_test.append(1*(test_output[i][j]>0.5))

In [194]:
test_output[0][0]

array([[0.3174966 , 0.16024208, 0.04908505, ..., 0.11100855, 0.13705626,
        0.14573294],
       [0.206088  , 0.03769097, 0.00603878, ..., 0.03638604, 0.04272217,
        0.06507322],
       [0.2889959 , 0.03596732, 0.00507131, ..., 0.02485043, 0.02645996,
        0.04582813],
       ...,
       [0.00282884, 0.00230673, 0.00124073, ..., 0.00042281, 0.00051597,
        0.00048792],
       [0.00283712, 0.00231522, 0.00124618, ..., 0.00042453, 0.00051758,
        0.00048909],
       [0.00284633, 0.00232342, 0.00125208, ..., 0.00042641, 0.00051913,
        0.0004904 ]], dtype=float32)

saves output files in the above created directory

In [226]:
#write output files
pred = np.append(pred, op,0)
for i in range(400):
    pred[i] = 1*(pred[i]>0.5)
    temp=pred[i]*test_stft[i]
    clean_istft = librosa.istft(temp.T,win_length=1024, hop_length = 512, length = test_load[i].shape[0])
    librosa.output.write_wav("test_outputs/test_clean_"+str(i)+".wav", clean_istft, sr=16000)

In [112]:
sess_a3_part2.close()